In [1]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [2]:
import tensorflow as tf
import tensorflow_addons as tfa
import os, shutil
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
original_dataset_dir = '/content/drive/MyDrive/data'
base_dir = '/content/drive/MyDrive/data'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [5]:
conv_base = tf.keras.applications.EfficientNetB2(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3))

31790344/31790344 [==============================] - 0s 0us/step


In [6]:
conv_base.summary()

Model: "efficientnetb2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 224, 224, 3)  0           ['normalization[0][0

In [54]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(8, activation='softmax'))

In [55]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb2 (Functional)  (None, 7, 7, 1408)       7768569   
                                                                 
 flatten_2 (Flatten)         (None, 68992)             0         
                                                                 
 dense_4 (Dense)             (None, 512)               35324416  
                                                                 
 dense_5 (Dense)             (None, 8)                 4104      
                                                                 
Total params: 43,097,089
Trainable params: 35,328,520
Non-trainable params: 7,768,569
_________________________________________________________________


In [56]:
conv_base.trainable = False

In [57]:
from keras.preprocessing.image import ImageDataGenerator

batch = 100

train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=5,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    shuffle=True)
label_map = (train_generator.class_indices)

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=batch,
        class_mode='categorical',
        shuffle=True)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    shuffle=False)

label_map

Found 5599 images belonging to 8 classes.
Found 1600 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


{'Засветы': 0,
 'Малая часть упаковки видна на фото': 1,
 'Не читабилен текст': 2,
 'Нормальные': 3,
 'Размытые': 4,
 'Фото не в том режиме': 5,
 'Фото полки где 4-5 товара и видно что фоткали не основ товар': 6,
 'Фото ценника': 7}

In [58]:
from tensorflow.keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])

In [59]:
train_stop = train_generator.samples//batch
validation_stop = validation_generator.samples//batch


In [ ]:
from keras.callbacks import LearningRateScheduler

In [60]:
%%time
history = model.fit(train_generator,
                    steps_per_epoch = train_stop,
                    epochs=15,
                    validation_data = validation_generator,
                    validation_steps = validation_stop)

Epoch 1/20
55/55 [==============================] - 325s 6s/step - loss: 5.3922 - acc: 0.1311 - val_loss: 2.8849 - val_acc: 0.1250
Epoch 2/20
55/55 [==============================] - 310s 6s/step - loss: 3.1012 - acc: 0.1304 - val_loss: 2.5645 - val_acc: 0.1250
Epoch 3/20
55/55 [==============================] - 306s 6s/step - loss: 3.0825 - acc: 0.1238 - val_loss: 2.5110 - val_acc: 0.1250
Epoch 4/20
55/55 [==============================] - 303s 6s/step - loss: 2.8802 - acc: 0.1251 - val_loss: 2.4032 - val_acc: 0.1281
Epoch 5/20
55/55 [==============================] - 304s 6s/step - loss: 2.8410 - acc: 0.1318 - val_loss: 3.2774 - val_acc: 0.1250
Epoch 6/20
55/55 [==============================] - 303s 6s/step - loss: 2.8192 - acc: 0.1344 - val_loss: 2.4950 - val_acc: 0.1250
Epoch 7/20
55/55 [==============================] - 302s 6s/step - loss: 2.7173 - acc: 0.1402 - val_loss: 2.5732 - val_acc: 0.1625
Epoch 8/20
55/55 [==============================] - 303s 6s/step - loss: 2.7468 - a

In [61]:
test = model.evaluate(test_generator)

8/8 [==============================] - 34s 4s/step - loss: 2.8991 - acc: 0.1250


In [62]:
model.save('/content/drive/MyDrive/data/ENetB2notS.h5')

In [63]:
import pandas as pd

In [64]:
x_test = test_generator.filepaths
y_test = test_generator.labels


In [65]:
pred = model.predict(test_generator)

8/8 [==============================] - 37s 4s/step


In [66]:
predicted_classes = np.argmax(pred, axis = 1)

In [67]:
df_marks = pd.DataFrame({'x_test': x_test,
                         'y_test': y_test,
                         'predicted_classes': predicted_classes})

In [68]:
df_marks.to_csv('/content/drive/MyDrive/data/file_pred.csv')

In [69]:
print(predicted_classes)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 

In [70]:
from sklearn.metrics import classification_report
report = classification_report(y_test, predicted_classes)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       100
           1       0.00      0.00      0.00       100
           2       0.12      1.00      0.22       100
           3       0.00      0.00      0.00       100
           4       0.00      0.00      0.00       100
           5       0.00      0.00      0.00       100
           6       0.00      0.00      0.00       100
           7       0.00      0.00      0.00       100

    accuracy                           0.12       800
   macro avg       0.02      0.12      0.03       800
weighted avg       0.02      0.12      0.03       800



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
